In [1]:
import pandas as pd
import copy
import pickle
from bertopic import BERTopic

/home/cattiaux/anaconda3/envs/wassati/lib/python3.9/site-packages/umap/distances.py:1063: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/home/cattiaux/anaconda3/envs/wassati/lib/python3.9/site-packages/umap/distances.py:1071: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/home/cattiaux/anaconda3/envs/wassati/lib/python3.9/site-packages/

In [2]:
df_old = pd.read_csv("../data/csv_files/schneider_processed_labelled_full.csv")

/tmp/ipykernel_14080/112462886.py:1: DtypeWarning: Columns (5,6,14,18) have mixed types. Specify dtype option on import or set low_memory=False.
  df_old = pd.read_csv("../data/csv_files/schneider_processed_labelled_full.csv")


In [3]:
import sys
sys.path.append('../')

from utils.schneider import countries_to_update, text_data_column, words_to_filter, replacements
from preprocessing.dataLoaders.schneider_data_loader import SchneiderDataLoader
from preprocessing.preprocessing import Preprocessor

df = pd.read_csv("../data/csv_files/schneider_2023.csv")
schneiderDataLoader = SchneiderDataLoader(df, countries_to_update)
preprocessing = Preprocessor(
    schneiderDataLoader, 
    text_data_column,
    words_to_filter,
    replacements
)

df_2023_preprocessed_full = preprocessing.preprocess()
# Filter rows based on 'non_empty_rows' column
df_2023_preprocessed = df_2023_preprocessed_full[df_2023_preprocessed_full['non_empty_rows']]

In [4]:
def load_bertopic_model(filename):
    """
    Load a BERTopic model and associated data from a file.
    
    :param filename: The name of the file to load the data from.
    :return: A tuple containing the loaded BERTopic model, topics, probs, and docs variables.
    """
    # Load the BERTopic model
    topic_model = BERTopic.load(filename)
    
    # Load the topics, probs, and docs variables
    with open(filename + '_data.pkl', 'rb') as f:
        topics, probs, embeddings, docs = pickle.load(f)
    
    return topic_model, topics, probs, embeddings, docs

topic_model, topics, probs, embeddings, docs = load_bertopic_model('../models/raw_keybert_bertopic_model')

def create_merged_model(docs, bertopic_model, topics_to_merge_dict, label_names_dict):
    """
    Create a new BERTopic model by merging topics from an existing model.

    This function takes as input a list of documents `docs`, an existing BERTopic model `bertopic_model`, a dictionary `topics_to_merge_dict` specifying which topics to merge, and a dictionary `label_names_dict` specifying the labels for the merged topics.

    The function creates a deep copy of the input BERTopic model and merges the specified topics using the `merge_topics` method. Then, it sets the topic labels for the merged model using the `set_topic_labels` method and the provided `label_names_dict`.

    The resulting merged BERTopic model is then returned.

    Parameters:
        docs (list): A list of documents used to fit the BERTopic model.
        bertopic_model (BERTopic): The input BERTopic model to be merged.
        topics_to_merge_dict (dict): A dictionary specifying which topics to merge. The keys are the topic numbers to be merged, and the values are the topic numbers into which they should be merged.
        label_names_dict (dict): A dictionary specifying the labels for the merged topics. The keys are the topic numbers, and the values are the corresponding labels.

    Returns:
        BERTopic: The resulting merged BERTopic model.
    """
    topic_model_merged = copy.deepcopy(bertopic_model)
    topic_model_merged.merge_topics(docs, topics_to_merge_dict)

    # Create a dictionary to match the aggregated name to their corresponding topic number
    mergedtopic_labels_dict = {i-1: item for i, item in enumerate(label_names_dict)}
    # Set topic labels for the aggregated model
    topic_model_merged.set_topic_labels(mergedtopic_labels_dict)

    return topic_model_merged### Création du modèle bertopic aggrégé pour topics finaux

# List of topics numbers. Each value of this list is a list that contains the topic number of the topics to join together
topics_to_merge = [ [42,3,0,13], #Delivery Deadlines : challenges and strategies involved in managing delivery deadlines in logistics operations. (vert)
                    [20,50,27], #Quotation and Pricing Strategies (vert bas)
                    [35,32], #Touch Panels and Screens (rouge, haut)
                    [40,36], #Frequency Converters : frequency converters used in industrial applications and the technical support provided by manufacturers and suppliers (rouge, suite)
                    [37,21,6,12,9,4,1,14,16,31,19], #“Automation Components” : hardware and software components used in industrial automation systems. (rouge centre)
                    [33,46,8], #Product Evaluation : evaluate the quality, affordability and reliability of products and services (rouge, fin)
                    [44,51,23,41,49,57,22], #Customer Support : Reliability and Quality in Customer Service and Support (bleu ciel)
                    [58,59], #Quick Customer Service (marron)
                    [38,10,26,52,39,43], #Problem Solving and Communication (focus on the importance of being efficient and precise when solving problems) (jaune)
                    [45,47,55,53,54], #Assistance and Guidance (noir)
                    [29,30,11,24], #Power Supply Issues (2e vert, haut)
                    [7,5,2,25,15,34,18,28,17], #Technical Support (2e vert, bas)
                    [48,56] #None : positive feedback (2e rouge)
]

label_names_postmerge = [
    "Outliers",
    "Automation Components",
    "Technical Support",
    "Delivery Deadlines",
    "Problem Solving & Comm",
    "Power Supply Issues",
    "Customer Support", #Reliability and Quality in Customer Service and Support
    "Product Evaluation",
    "Pricing", #Quotation and Pricing Strategies
    "Assistance", #Assistance and Guidance
    "Touch Screens", #Touch Panels and Screens
    "Frequency Converters",
    "Positive feedback",
    "Quick Customer Service"
    ]

# Set the topic names for the new aggregated topic
# It must match the order from the topics_to_merge list
label_names = ["Delivery Deadlines",
    "Pricing", #Quotation and Pricing Strategies
    "Touch Screens", #Touch Panels and Screens
    "Frequency Converters",
    "Automation Components",
    "Product Evaluation",
    "Customer Support", #Reliability and Quality in Customer Service and Support
    "Quick Customer Service",
    "Problem Solving & Comm",
    "Assistance", #Assistance and Guidance
    "Power Supply Issues",
    "Technical Support",
    "positive feedback"]

topic_labels_dict = {}
topic_labels_dict[-1]="Outliers"
for i in range(len(topics_to_merge)):
    for topic in topics_to_merge[i]:
        topic_labels_dict[topic] = label_names[i]

# Create a new merged bertopic model 
topic_model_merged = create_merged_model(docs, topic_model, topics_to_merge, label_names_postmerge)

# topic_model_merged = copy.deepcopy(topic_model)
# topic_model_merged.set_topic_labels(topic_labels_dict)
# topic_model_merged.merge_topics(docs, topics_to_merge)

/home/cattiaux/anaconda3/envs/wassati/lib/python3.9/site-packages/bertopic/vectorizers/_ctfidf.py:69: RuntimeWarning: divide by zero encountered in divide
  idf = np.log((avg_nr_samples / df)+1)


In [5]:
df_2023_preprocessed.reset_index(drop=True, inplace=True)
new_topics, new_probs = topic_model_merged.transform(df_2023_preprocessed['processed_data'])

# Now, you can merge the new dataframe with the old one
# df_labelled = pd.concat([df_labelled, df_2023_preprocessed])

In [6]:
# Get the topic and keywords for the new data
new_topic_keywords = {}
n = 10  # Number of keywords to include
for topic in topic_model_merged.get_topics().keys():
    topic_words = [word for word, _ in topic_model_merged.get_topic(topic)[:n]]
    new_topic_keywords[topic] = ", ".join(topic_words)

# Add the topics and keywords to the new dataframe
df_2023_preprocessed['topic'] = new_topics
df_2023_preprocessed['keywords'] = df_2023_preprocessed['topic'].map(new_topic_keywords)

# Add the label column to the new dataframe
mergedtopic_labels_dict = {i-1: item for i, item in enumerate(label_names_postmerge)}
df_2023_preprocessed['label'] = df_2023_preprocessed['topic'].map(mergedtopic_labels_dict)

# Get a list of the topics in the correct order, excluding topic -1
topic_order = [topic for topic in topic_model_merged.get_topics().keys() if topic != -1]

# Create a dictionary where the keys are the topic labels and the values are the probabilities
new_proba_dict = []
for proba in new_probs:
    new_proba_dict.append(dict(sorted(zip(topic_order, proba), key=lambda item: item[1], reverse=True)))
    
# Add the proba_dict column to the new dataframe
df_2023_preprocessed['proba_dict'] = new_proba_dict

In [7]:
# Now we can append the two dataframes, the 2023 data not null with the null rows from 2023_full
df_final = df_2023_preprocessed.append(df_2023_preprocessed_full[df_2023_preprocessed_full['non_empty_rows'] == False], ignore_index=True)
df_final['Response Date'] = pd.to_datetime(df_final['Response Date'])
df_final['year_month'] = df_final['Response Date'].dt.to_period('M').dt.to_timestamp()
df_final['Market Segment'] = df_final['Market Segment'].fillna('Unknown')

In [10]:
set(df_final.columns) - set(df_old.columns)

{'Survey Type', 'Year'}

In [8]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline

### Analyse de Sentiments simple : positif - négatif - neutre

def load_model_huggingface(model_name, task, problem_type=None, **kwargs):
    """
    This function loads a model and tokenizer from a given model name, then creates a pipeline to perform a specified task.

    Args:
        model_name (str): The name of the model to load.
        task (str): The type of task to perform with the pipeline.
        problem_type (str): The type of problem to solve ("multi_label_classification" for multi-label tasks).
        **kwargs: Additional arguments to pass to the pipeline.

    Returns:
        pipeline: A pipeline configured to perform the specified task with the loaded model and tokenizer.
    """
    model = AutoModelForSequenceClassification.from_pretrained(model_name, problem_type=problem_type)
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    classifier = pipeline(task, model=model, tokenizer=tokenizer, **kwargs)
    return classifier

def add_single_label_predictions(df, predictions, predicted_column_name):
    """
    This function merges the DataFrame of single-label predictions with the original DataFrame.

    Args:
        df (pd.DataFrame): The original DataFrame.
        predictions (list): The list of predictions. Each prediction is a dictionary containing a 'label' and a 'score'.
        predicted_column_name (str): The name of the column to be added to the DataFrame.

    Returns:
        pd.DataFrame: The original DataFrame with added columns for the predicted labels and their scores.
    """
    predicted_df = df
    # Convert the predictions to a DataFrame
    prediction_results = pd.DataFrame(predictions)
    prediction_results.rename(columns={'label': predicted_column_name}, inplace=True)
    # # Reset the indices of the DataFrames (if necessary)
    # df.reset_index(drop=True, inplace=True)
    # prediction_results.reset_index(drop=True, inplace=True)
    # Merge the original DataFrame with the prediction results
    df_predicted = pd.concat([predicted_df, prediction_results], axis=1)
    return df_predicted

def add_multi_label_predictions(df, predictions, predicted_column_name):
    """
    This function adds a new column with multi-label predictions to the DataFrame and also adds two more columns for 
    the best label and its score.

    Args:
        df (pd.DataFrame): The original DataFrame.
        predictions (list): The list of predictions. Each prediction is a list of dictionaries, where each dictionary 
                            contains a 'label' and a 'score'.
        predicted_column_name (str): The name of the column to be added to the DataFrame.

    Returns:
        pd.DataFrame: The original DataFrame with added columns for the predicted labels and their scores, as well as 
                      columns for the best label and its score.
    """
    predicted_df = df
    # Keep the original predictions as they are (a list of dictionaries) and add them to the DataFrame as a new column
    predicted_df[predicted_column_name] = predictions
    # Add columns for the best label and its score
    predicted_df[f'best_{predicted_column_name}'] = predicted_df[predicted_column_name].apply(lambda x: max(x.keys(), key=lambda k: x[k]) if x else None)
    predicted_df[f'best_{predicted_column_name}_score'] = predicted_df[predicted_column_name].apply(lambda x: x[max(x.keys(), key=lambda k: x[k])] if x else None)
    return predicted_df

def make_predictions_df(classifier, df, predicted_column_name):
    """
    This function makes predictions on a DataFrame of documents using a given classifier. It adds the predictions to 
    the DataFrame as new columns. If the classifier is for single-label classification, it adds one column for the 
    predicted label and one for the score. If the classifier is for multi-label classification, it adds one column 
    with a dictionary of label-score pairs for each document, and two additional columns for the best label and its score.

    Args:
        classifier (pipeline): The Hugging Face pipeline object for making predictions.
        df (pd.DataFrame): The DataFrame containing the documents to make predictions on. It must have a 'processed_data' 
                           column with the preprocessed text of each document.
        predicted_column_name (str): The name of the column to be added to the DataFrame for the predictions.

    Returns:
        pd.DataFrame: The original DataFrame with added columns for the predictions.
    """
    # Get the list of documents from the DataFrame
    docs = df["processed_data"].tolist()
    # Get predictions
    predictions = classifier(docs)
    
    # Check if predictions is a list of dictionaries (single-label case)
    if isinstance(predictions, list) and isinstance(predictions[0], dict):
        df_predicted = add_single_label_predictions(df, predictions, predicted_column_name)
    
    # Multi-label case
    elif isinstance(predictions, list) and isinstance(predictions[0], list):
        df_predicted = add_multi_label_predictions(df, predictions, predicted_column_name)

    return df_predicted

classifier = load_model_huggingface("cardiffnlp/twitter-roberta-base-sentiment-latest", "text-classification", max_length=512, truncation=True)
predicted_df = make_predictions_df(classifier, df_final, 'sentiment_label')

predictions = []
for review in [[x] for x in df_final.processed_data.tolist()]:
      # Effectuer une prédiction pour le document en utilisant le classificateur
    prediction = classifier(review)
      # Ajouter la prédiction à la liste des prédictions
    predictions.append(prediction)

# Création d'un DataFrame à partir des résultats de prédiction
res = pd.DataFrame([item for sublist in predictions for item in sublist])
df_pred = pd.concat([df_final, res], axis=1)

# The model created a column 'label' to store the prediction, BUT the df already had one column 'label', then :
# Rename the second 'label' column to 'sentiment_label'
cols = df_pred.columns.tolist()
cols[len(cols) - 1 - cols[::-1].index('label')] = 'sentiment_label'
df_pred.columns = cols

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

Xformers is not installed correctly. If you want to use memory_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.


In [9]:
import torch
import ast

### Analyse de Sentiments approfondie : par 28 émotions

# en multilabel, avec GPU et par batch pour accélerer le traitement

# Check if a CUDA-enabled GPU is available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model_name = "SamLowe/roberta-base-go_emotions"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, problem_type="multi_label_classification", max_length=512)

# Move the model to the GPU
model = model.to(device)

# Set the batch size
batch_size = 2

# Create a list of label names
label_emotions = ['admiration', 'amusement', 'anger', 'annoyance', 'approval', 'caring', 'confusion', 'curiosity', 'desire', 'disappointment', 'disapproval', 'disgust', 'embarrassment', 'excitement', 'fear', 'gratitude', 'grief', 'joy', 'love', 'nervousness', 'optimism', 'pride', 'realization', 'relief', 'remorse', 'sadness', 'surprise','neutral']

# Initialize lists to store the predicted labels and scores
predicted_labels = []
predicted_scores = []

df_emotion = df_pred
# Iterate over the rows of the DataFrame in batches
for i in range(0, len(df_emotion), batch_size):
    batch = df_emotion[i:i+batch_size]
    texts = batch['processed_data'].tolist()
    inputs = tokenizer(texts, return_tensors="pt", padding=True, truncation=True)
    
    # Move the inputs to the GPU
    inputs = {k: v.to(device) for k, v in inputs.items()}
    
    outputs = model(**inputs)
    probs = outputs.logits.sigmoid().detach().cpu().numpy()
    
    # Apply a threshold to the probabilities to get the predicted labels
    threshold = 0.5
    labels = [[label_emotions[i] for i, prob in enumerate(prob_row) if prob > threshold] for prob_row in probs]
    
    # Store the predicted labels and scores
    predicted_labels.extend(labels)
    scores = [{label_emotions[i]: prob for i, prob in enumerate(prob_row)} for prob_row in probs]
    # predicted_scores.extend(probs.tolist())
    predicted_scores.extend(scores)

# Add the predicted labels and scores as new columns in the DataFrame
df_emotion['predicted_labels'] = predicted_labels
df_emotion['predicted_scores'] = predicted_scores

# # TO DO when reading the df, since the list columns will be read as full string and not list...
# # Convert the string values in the 'predicted_labels' column into lists
# df_emotion['predicted_labels'] = df_emotion['predicted_labels'].apply(lambda x: ast.literal_eval(x))
# # Convert the string values in the 'predicted_scores' column into dictionaries
# df_emotion['predicted_scores'] = df_emotion['predicted_scores'].apply(lambda x: ast.literal_eval(x))

In [10]:
def convert_dict(input_dict, conversion_type='keys_to_values'):
    """
    This function converts between a key-to-value dictionary and a value-to-keys dictionary.

    Args:
        input_dict (dict): The input dictionary. If conversion_type is 'keys_to_values', this should be a dictionary where 
                           the keys are the original keys and the values are the corresponding values. If conversion_type is 
                           'values_to_keys', this should be a dictionary where the keys are the original values and the values 
                           are lists of keys for each value.
        conversion_type (str): The type of conversion to perform. Can be either 'keys_to_values' or 'values_to_keys'.

    Returns:
        dict: The converted dictionary. If conversion_type is 'keys_to_values', this will be a dictionary where the keys 
              are the original values and the values are lists of keys for each value. If conversion_type is 'values_to_keys', 
              this will be a dictionary where the keys are the original keys and the values are the corresponding values.
    """
    if conversion_type == 'keys_to_values':
        output_dict = {}
        for key, value in input_dict.items():
            if value not in output_dict:
                output_dict[value] = []
            output_dict[value].append(key)
    elif conversion_type == 'values_to_keys':
        output_dict = {key: value for value, keys in input_dict.items() for key in keys}
    else:
        raise ValueError("Invalid conversion_type. Must be either 'keys_to_values' or 'values_to_keys'.")
    
    return output_dict

def add_grouping_column(df, key_column, group_dict, group_column_name):
    """
    This function adds a new column to a DataFrame with the group name of each key.

    Args:
        df (pd.DataFrame): The original DataFrame.
        key_column (str): The name of the column in df that contains the keys.
        group_dict (dict): A dictionary where the keys are the original keys and the values are the corresponding group names.
        group_column_name (str): The name of the new column to be added to df for the groups.

    Returns:
        pd.DataFrame: The original DataFrame with an added column for the groups.
    """
    # df[group_column_name] = df[key_column].map(group_dict)
    df[group_column_name] = df[key_column].apply(lambda labels: [group_dict[label] for label in labels if label in group_dict])
    return df

# transform the multilabel results from the emotions analysis in single label
# Define the groups
positive_emotions = ['admiration','approval','gratitude','caring','realization','joy','optimism','love','excitement','amusement','relief']
negative_emotions = ['disappointment','disapproval','annoyance','confusion','nervousness','fear','sadness','remorse','disgust','embarrassment','anger']
neutral_emotions = ['neutral','desire','surprise','curiosity']
# Create a dictionary where the keys are the group names and the values are the lists of labels
groups = {"positive": positive_emotions, "negative": negative_emotions, "neutral": neutral_emotions}
# Convert the groups to a group_dict
group_dict = convert_dict(groups, conversion_type='values_to_keys')

# Now you can use group_dict with add_grouping_column
# df_emotion = add_grouping_column(df_emotion, "predicted_labels", group_dict, "sentiment_from_emotion_label")

# Create a new column 'single_emotion_label' that contains the label with the highest score
df_emotion['single_emotion_label'] = df_emotion['predicted_scores'].apply(lambda x: max(x, key=x.get))
# Create a new column 'sentiment' that contains the sentiment of the emotion in the 'single_emotion_label' column
df_emotion['single_sentiment_from_emotion'] = df_emotion['single_emotion_label'].apply(lambda x: 'positive' if x in positive_emotions else ('negative' if x in negative_emotions else ('neutral' if x in neutral_emotions else 'unknown')))

# df3.to_csv("/media/cattiaux/DATA/Wassati/team_data/schneider/df_all_labelled.csv")

In [32]:
test = df_emotion[["predicted_labels", 'predicted_scores', 'sentiment_label', 'topic','label','keywords','single_emotion_label','single_sentiment_from_emotion','sentiment_from_emotion_label']].head()
test

,predicted_labels,predicted_scores,sentiment_label,topic,label,keywords,single_emotion_label,single_sentiment_from_emotion,sentiment_from_emotion_label
0,[neutral],"{'admiration': 0.001663754, 'amusement': 0.000...",neutral,-1.0,Outliers,"customer service, delivery time, sales, techni...",neutral,neutral,[neutral]
1,[],"{'admiration': 0.104325175, 'amusement': 0.000...",neutral,-1.0,Outliers,"customer service, delivery time, sales, techni...",neutral,neutral,[]
2,[neutral],"{'admiration': 0.0010179136, 'amusement': 0.00...",negative,2.0,Delivery Deadlines,"delivery time, delays, delivery date, deliveri...",neutral,neutral,[neutral]
3,[admiration],"{'admiration': 0.84221095, 'amusement': 0.0011...",positive,1.0,Technical Support,"technical support, response time, customer ser...",admiration,positive,[positive]
4,[neutral],"{'admiration': 0.0072880113, 'amusement': 0.00...",positive,7.0,Pricing,"pricing, prices, discount, quotation, orders, ...",neutral,neutral,[neutral]


In [12]:
df_emotion["allComment"] = df_emotion.processed_data
df_2023_full =df_emotion.append(df_old, ignore_index=True)
len(df_2023_full)

62843

In [20]:
df_emotion =  df_emotion[df_emotion.columns.difference(['Survey Type', 'Year'])]
df_schneider_full = df_old.append(df_emotion, ignore_index=True)
df_schneider_full.to_csv("../data/csv_files/schneider_all_processed_labelled_full.csv", index=False)